# Import libraries

In [3]:
import os
import pandas as pd
import re

# 1- Extract sheets from the excel files

In [22]:
# Define input and output folders
input_folder = "data"
velocity_output_folder = "extracted_data\\velocity"
acceleration_output_folder = "extracted_data\\acceleration"

# Make sure output folders exist
os.makedirs(input_folder, exist_ok=True)
os.makedirs(velocity_output_folder, exist_ok=True)
os.makedirs(acceleration_output_folder, exist_ok=True)

# Loop through each excel file
for file_name in os.listdir(input_folder):
    if file_name.endswith(".xlsx"):

        file_path = os.path.join(input_folder, file_name)
        xls = pd.ExcelFile(file_path)

        # Read the 'Segment Velocity' and 'Segment Acceleration' sheets
        velocity_df = xls.parse('Segment Velocity')
        acceleration_df = xls.parse('Segment Acceleration')

        # Create output paths
        velocity_output_path = os.path.join(
            velocity_output_folder, f"{file_name.split('.')[0]}-Velocity.csv")

        acceleration_output_path = os.path.join(
            acceleration_output_folder, f"{file_name.split('.')[0]}-Acceleration.csv")

        # Save the extracted data to a new csv file
        velocity_df.to_csv(velocity_output_path, index=False)
        acceleration_df.to_csv(acceleration_output_path, index=False)

        print(f"Processed and saved: {file_name}")

Processed and saved: MVN-J-Boning-64-001.xlsx
Processed and saved: MVN-J-Boning-64-002.xlsx
Processed and saved: MVN-J-Boning-64-003.xlsx
Processed and saved: MVN-J-Boning-64-004.xlsx
Processed and saved: MVN-J-Boning-64-005.xlsx
Processed and saved: MVN-J-Boning-64-006.xlsx
Processed and saved: MVN-J-Boning-79-001.xlsx
Processed and saved: MVN-J-Boning-90-001.xlsx
Processed and saved: MVN-J-Boning-90-002.xlsx
Processed and saved: MVN-J-Boning-90-003.xlsx
Processed and saved: MVN-J-Boning-90-004.xlsx
Processed and saved: MVN-J-Slicing-64-001.xlsx
Processed and saved: MVN-J-Slicing-73-001.xlsx
Processed and saved: MVN-J-Slicing-87-001.xlsx
Processed and saved: MVN-S-Boning-63-001.xlsx
Processed and saved: MVN-S-Boning-63-002.xlsx
Processed and saved: MVN-S-Boning-63-003.xlsx
Processed and saved: MVN-S-Boning-76-001.xlsx
Processed and saved: MVN-S-Boning-76-002.xlsx
Processed and saved: MVN-S-Boning-89-001.xlsx
Processed and saved: MVN-S-Boning-89-002.xlsx
Processed and saved: MVN-S-Boni

# 2 - Merge velocity and acceleration datasets

## 2.1 - Define folders

In [23]:
# Define input and output folders
velocity_folder = "extracted_data\\velocity"
acceleration_folder = "extracted_data\\acceleration"

# Make sure output folders exist
os.makedirs(velocity_folder, exist_ok=True)
os.makedirs(velocity_folder, exist_ok=True)

# Define folder for saving merged files
merged_output_folder = 'merged_data'
os.makedirs(merged_output_folder, exist_ok=True)  # Create the merged folder if it doesn't exist

## Handle edge cases

In [25]:
# Function to assign labels based on the Marker sheet
def assign_labels(frame_df, marker_df):
    labels = []
    
    # Loop through the marker dataframe
    for index, row in marker_df.iterrows():
        # Extract frame range and label number
        frame_range = row['Frame']  # e.g., '0-195'
        label = int(row['Label'].split()[0])  # Extract number from '3 - Reaching'
        
        # Split the frame range into start and end
        start_frame, end_frame = map(int, frame_range.split('-'))
        
        # Assign label to each frame in the range
        for frame in range(start_frame, end_frame + 1):
            labels.append((frame, label))
    
    # Convert the list of frame-label pairs into a DataFrame
    label_df = pd.DataFrame(labels, columns=['Frame', 'Label'])
    
    # Merge the labels with the original frame data
    merged_df = pd.merge(frame_df, label_df, on='Frame', how='left')
    
    return merged_df


In [33]:
def preprocess_files(velocity_file, acceleration_file, marker_file=None):
    # Load the velocity and acceleration files
    velocity_df = pd.read_csv(os.path.join(velocity_folder, velocity_file))
    acceleration_df = pd.read_csv(os.path.join(acceleration_folder, acceleration_file))

    # Handle boning-90-003: Copy Label from velocity to acceleration
    if 'boning-90-003' in velocity_file.lower():
        print(f"Fixing missing Label in acceleration sheet for: {acceleration_file}")
        acceleration_df['Label'] = velocity_df['Label']

    # Handle slicing-64-001: Rename Marker to Label
    if 'slicing-64-001' in velocity_file.lower():
        print(f"Renaming 'Marker' to 'Label' for: {velocity_file} and {acceleration_file}")
        velocity_df.rename(columns={'Marker': 'Label'}, inplace=True)
        acceleration_df.rename(columns={'Marker': 'Label'}, inplace=True)

    # Handle slicing-87-001: Add Label from Marker sheet
    if 'slicing-87-001' in velocity_file.lower():
        if marker_file is not None:
            print(f"Adding Label from Marker sheet for: {velocity_file} and {acceleration_file}")
            marker_df = pd.read_csv(marker_file)
            velocity_df = assign_labels(velocity_df, marker_df)
            acceleration_df = assign_labels(acceleration_df, marker_df)

    # Save the preprocessed data back to CSV files (overwrite or in new location)
    velocity_df.to_csv(os.path.join(velocity_folder, velocity_file), index=False)
    acceleration_df.to_csv(os.path.join(acceleration_folder, acceleration_file), index=False)

    return velocity_file, acceleration_file

In [35]:
velocity_files_to_preprocess = [
    'MVN-J-boning-90-003-Velocity.csv',
    'MVN-J-slicing-64-001-Velocity.csv',
    'MVN-J-slicing-87-001-Velocity.csv'
]

acceleration_files_to_preprocess = [
    'MVN-J-boning-90-003-Acceleration.csv',
    'MVN-J-slicing-64-001-Acceleration.csv',
    'MVN-J-slicing-87-001-Acceleration.csv'
]


# Preprocess the files
for velocity_file, acceleration_file in zip(velocity_files_to_preprocess, acceleration_files_to_preprocess):

    marker_file = None
    if 'slicing-87-001' in velocity_file.lower():
        marker_file = 'data\\MVN-J-Slicing-87-001-Marker.csv'  # Adjust this path to the actual Marker file
        
    preprocess_files(velocity_file, acceleration_file, marker_file)

Fixing missing Label in acceleration sheet for: MVN-J-boning-90-003-Acceleration.csv
Renaming 'Marker' to 'Label' for: MVN-J-slicing-64-001-Velocity.csv and MVN-J-slicing-64-001-Acceleration.csv
Adding Label from Marker sheet for: MVN-J-slicing-87-001-Velocity.csv and MVN-J-slicing-87-001-Acceleration.csv


## 2.2 - Function to merge acceleration and velocity

In [37]:
# Merge the velocity and acceleration data for each file
def merge_acc_vel(velocity_file, acceleration_file):
    # Read the velocity and acceleration data
    velocity_df = pd.read_csv(os.path.join(velocity_folder, velocity_file))
    acceleration_df = pd.read_csv(os.path.join(acceleration_folder, acceleration_file))

    # Check if the common columns are present in both dataframes
    common_cols = ['Frame', 'Label']
    if not all(column in velocity_df.columns and column in acceleration_df.columns for column in common_cols):
        print(f"Error: Common columns not found in {velocity_file} and {acceleration_file}. Skipping...")
        return

    # Merge the velocity and acceleration data on the common columns
    merged_df = pd.merge(velocity_df, acceleration_df, on=common_cols, suffixes=('_Vel', '_Acc'))

    # Save the merged dataframe as a new CSV file
    merged_filename = velocity_file.replace('-Velocity', '-Merged')
    merged_df.to_csv(os.path.join(merged_output_folder, merged_filename), index=False)

    print(f"Successfully merged and saved: {merged_filename}")

## 2.3 - Perform merging

In [38]:
# Get the list of all velocity and acceleration files
velocity_files = [f for f in os.listdir(velocity_folder) if f.endswith(".csv")]
acceleration_files = [f for f in os.listdir(acceleration_folder) if f.endswith(".csv")]

for velocity_file in velocity_files:
    # Find the corresponding acceleration file by replacing "Velocity" with "Acceleration"
    acceleration_file = velocity_file.replace("Velocity", "Acceleration")
    
    if acceleration_file in acceleration_files:
        merge_acc_vel(velocity_file, acceleration_file)
    else:
        print(f"Corresponding acceleration file not found for {velocity_file}. Skipping...")

Successfully merged and saved: MVN-J-Boning-64-001-Merged.csv
Successfully merged and saved: MVN-J-Boning-64-002-Merged.csv
Successfully merged and saved: MVN-J-Boning-64-003-Merged.csv
Successfully merged and saved: MVN-J-Boning-64-004-Merged.csv
Successfully merged and saved: MVN-J-Boning-64-005-Merged.csv
Successfully merged and saved: MVN-J-Boning-64-006-Merged.csv
Successfully merged and saved: MVN-J-Boning-79-001-Merged.csv
Successfully merged and saved: MVN-J-Boning-90-001-Merged.csv
Successfully merged and saved: MVN-J-Boning-90-002-Merged.csv
Successfully merged and saved: MVN-J-Boning-90-003-Merged.csv
Successfully merged and saved: MVN-J-Boning-90-004-Merged.csv
Successfully merged and saved: MVN-J-Slicing-64-001-Merged.csv
Successfully merged and saved: MVN-J-Slicing-73-001-Merged.csv
Successfully merged and saved: MVN-J-Slicing-87-001-Merged.csv
Successfully merged and saved: MVN-S-Boning-63-001-Merged.csv
Successfully merged and saved: MVN-S-Boning-63-002-Merged.csv
Succe

### Verify merging

In [43]:
df = pd.read_csv('merged_data\\MVN-J-Slicing-87-001-Merged.csv')

df.columns

print ("YES" if 'Label' in df.columns else "NO")

print (df['Label'].unique())

YES
[3 5 4 8 2 7]


# 3 - Add knife category column

In [55]:
"""
Extracts the knife sharpness value from the filename and categorizes it.
:param file_name: String, e.g., 'MVN-J-Boning-64-001_Acceleration.xlsx'
:return: Categorical label ('Blunt', 'Medium', 'Sharp')
"""
# Categorize knife sharpness
def add_sharpness_and_shift(file_name, df):
    # Extract knife sharpness factor from filename (e.g., '64' in 'MVN-J-Boning-64-001')
    knife_sharpness = int(file_name.split('-')[3])
    
    # Convert knife sharpness into categorical value
    if knife_sharpness >= 85:
        sharpness_category = 2  # Sharp
    elif 70 <= knife_sharpness <= 84:
        sharpness_category = 1  # Medium
    else:
        sharpness_category = 0  # Blunt
    
    # Add new column for sharpness category
    df['Knife_Sharpness_Category'] = sharpness_category

    # Extract the shift indicator from the filename (e.g., '001' in 'MVN-J-Boning-64-001')
    shift_indicator = file_name.split('-')[4]  # '001', '002', etc.
    
    # Add new column for the shift indicator
    df['Shift'] = shift_indicator
    
    return df

input_folder = "merged_data"
output_folder = "categorized_data"

os.makedirs(output_folder, exist_ok=True)  # Ensure the output folder exists

for file_name in os.listdir(input_folder):  
    if file_name.endswith(".csv"):

        file_path = os.path.join(input_folder, file_name)

        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)

        # Apply knife sharpness conversion function
        df = add_sharpness_and_shift(file_name, df)

        # Remove the "merged" annotation from the filename
        new_file_name = file_name.replace("-Merged", "")  # Replace "Merged" with an empty string

        # Save the updated DataFrame to a new CSV file with the knife sharpness column
        output_file_path = os.path.join(output_folder, new_file_name)
        df.to_csv(output_file_path, index=False)
        print(f"Processed and saved: {output_file_path}")

Processed and saved: categorized_data\MVN-J-Boning-64-001.csv
Processed and saved: categorized_data\MVN-J-Boning-64-002.csv
Processed and saved: categorized_data\MVN-J-Boning-64-003.csv
Processed and saved: categorized_data\MVN-J-Boning-64-004.csv
Processed and saved: categorized_data\MVN-J-Boning-64-005.csv
Processed and saved: categorized_data\MVN-J-Boning-64-006.csv
Processed and saved: categorized_data\MVN-J-Boning-79-001.csv
Processed and saved: categorized_data\MVN-J-Boning-90-001.csv
Processed and saved: categorized_data\MVN-J-Boning-90-002.csv
Processed and saved: categorized_data\MVN-J-Boning-90-003.csv
Processed and saved: categorized_data\MVN-J-Boning-90-004.csv
Processed and saved: categorized_data\MVN-J-Slicing-64-001.csv
Processed and saved: categorized_data\MVN-J-Slicing-73-001.csv
Processed and saved: categorized_data\MVN-J-Slicing-87-001.csv
Processed and saved: categorized_data\MVN-S-Boning-63-001.csv
Processed and saved: categorized_data\MVN-S-Boning-63-002.csv
Proce

In [54]:
# Test the outcome
df = pd.read_csv('categorized_data\\MVN-J-Slicing-87-001.csv')

print(df['Knife_Sharpness_Category'].unique())

print(df.shape)

[2]
(54721, 141)


# 4 - Merge different shift datasets for each knife sharpness and person

## Function to merge multiple CSV files for the same knife sharpness and activity

In [ ]:
# Function to merge multiple CSV files for the same knife sharpness and activity
def merge_files_for_activity_and_sharpness(file_list, output_file):
    merged_df = pd.concat([pd.read_csv(file) for file in file_list], ignore_index=True)
    merged_df.to_csv(output_file, index=False)
    print(f"Merged file saved as {output_file}")

## Group the files by activity and knife sharpness

In [ ]:
# Group the files by activity and knife sharpness
def group_files_by_activity_and_sharpness(files):
    grouped_files = {}
    for file in files:
        # Extract knife sharpness factor (64, 79, etc.) and activity (Boning, Slicing) from the filename
        activity = file.split('-')[2]  # Extract activity (Boning, Slicing)
        sharpness = file.split('-')[3]    # Extract the knife sharpness factor
        
        # Use both activity and sharpness as keys for grouping
        key = f"{activity}_{sharpness}"
        if key not in grouped_files:
            grouped_files[key] = []
        grouped_files[key].append(file)
    
    return grouped_files

## Main function to process and merge files for each person

In [ ]:
# Main function to process and merge files for each person
def process_and_merge_person_files(input_folder, output_folder, person_indicator):
    os.makedirs(output_folder, exist_ok=True)  # Ensure the output folder exists
    
    # Filter files for the specified person (J or S)
    person_files = [os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.startswith(f"MVN-{person_indicator}")]

    # Group the files by activity and knife sharpness
    grouped_files = group_files_by_activity_and_sharpness(person_files)

    # Merge files for each group (e.g., Boning-64, Slicing-87)
    for key, files in grouped_files.items():
        output_filename = os.path.join(output_folder, f"{person_indicator}_{key}_merged.csv")
        merge_files_for_activity_and_sharpness(files, output_filename)

In [ ]:
# Example usage to merge files for Person 1 (J) and Person 2 (S)
input_folder = "path_to_your_data_folder"  # Folder containing all the raw CSV files
output_folder_person1 = "path_to_output_person1"  # Folder to save merged files for Person 1
output_folder_person2 = "path_to_output_person2"  # Folder to save merged files for Person 2

# Merge files for Person 1 (J)
process_and_merge_person_files(input_folder, output_folder_person1, "J")

# Merge files for Person 2 (S)
process_and_merge_person_files(input_folder, output_folder_person2, "S")

# 5 - Convert to 1-minute interval 